In [1]:
#from transformers import pipeline
import nltk
from nltk import sent_tokenize
#import torch
from glob import glob
import pandas as pd 
import numpy  as pd 


In [ ]:
nltk.download('punkt')

#load Model


In [ ]:
model_name = "facebook/bart-large-mnli"
device = 0 if torch.cuda.is_available() else 'cpu'

In [ ]:
def load_model(device):
    theme_classifier = pipeline(
        "zero-shot-classification",
        model = model_name,
        device = device
    )
    return theme_classifier

In [ ]:
theme_classifier = load_model(device)

In [ ]:
theme_list = ["friendship" , "hope","sacrifice" , "battle","self development","betrayal","love" , "dialogue"]


In [ ]:
theme_classifier(
    "I give him a right hook hten a left job",
    theme_list,
    multi_label = True
)

#load Dataset

In [ ]:
" ".join(lines[:10])

In [ ]:
def load_subtitles_dataset(dataset_path):
    subtitles_paths = glob(dataset_path+'/*.ass') 
    
    scripts= []
    episode_num=[]
    
    
    for path in subtitles_paths :
       with open (path ,'r') as file:
          lines = file.readlines()
          lines = lines[27:]
          lines = [",".join(line.split(',')[9:])  for line in lines ]
    
       lines =  [line.replace('\\N' , ' ') for line in lines ]
       script = " ".join(lines)
       
       episode = int(path.split('-')[-1].split('.')[0])
       
       scripts.append(script)
       episode_num.append(episode)
       
    df = pd.DataFrame.from_dict({"episode":episode_num , "script" : scripts})
    return df      

In [ ]:
dataset_path = "../data/subtitles"
df = load_subtitles_dataset(dataset_path)



In [ ]:
df.head()

#Run Model

In [ ]:
script = df.iloc[0]['script']



In [ ]:
script

In [ ]:
script_sentances = sent_tokenize(script)
script_sentances [:3]

In [ ]:
#Batch Sentence 
sentance_batch_size = 20 
script_batches = []
for index in range (0 , len(script_sentances),sentance_batch_size):
    sent = " ".join(script_sentances[index:index+sentance_batch_size])
    script_batches.append(sent) 
   

In [ ]:
script_batches[:2]

In [ ]:
theme_output = theme_classifier(
    script_batches[:2],
    theme_list,
    multi_label=True
)

In [ ]:
theme_output

In [ ]:
themes = {}
for output in theme_output :
    for label,score in zip (output['labels'],output['scores']):
        if label not in themes  :
            themes[label]=[]    
        themes[label].append(score)

        


In [ ]:
themes

In [ ]:
def get_themes_inference(script):
    script_sentances = sent_tokenize(script)
    
    #Batch sentances
    sentances_batch_size = 20 
    script_batches = []
    for index in range (0 , len(script_sentances),sentance_batch_size):
    sent = " ".join(script_sentances[index:index+sentance_batch_size])
    script_batches.append(sent) 
    
    
    #run Model  
    theme_output = theme_classifier(
       script_batches,
       theme_list,
       multi_label=True
)
    #Wrangle Output 
    themes = {}
    for output in theme_output :
        for label,score in zip (output['labels'],output['scores']):
           if label not in themes  :
              themes[label]=[]    
           themes[label].append(score)
    themes = {key :np.mean(np.array(value)) for key,valu in themes.items() }      
    
    return themes   

In [ ]:
df = df.head(2)


In [ ]:
output_themes = df ['script'].apply(get_themes_inference )


In [ ]:
output_themes    

In [ ]:
theme_df =pd.DataFrame(output_themes.tolist())



In [ ]:
theme_df

In [ ]:
df

In [ ]:
df[theme_df.columns] = theme_df
df

# Visualize Output
 

In [ ]:
df = df.drop('dialoue',axis=1)


In [ ]:
df

In [ ]:
theme_output =  df.drop(['episode','script'],axis=1).sum().reset_index()
theme_output.culomns['theme','score']
theme_output


In [1]:
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
sns.barplot(data = theme_output, x ="theme" , y = "score")
plt.xticks(rotation=45)
plt.show()  

NameError: name 'theme_output' is not defined